<a href="https://colab.research.google.com/github/SunForLife/raiffeisen/blob/master/home_credit_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
import json
import zipfile
import os
!mkdir /root/.kaggle
!echo '' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!kaggle config path -p /root
!kaggle competitions download -c home-credit-default-risk
for file in os.listdir():
    if not file.endswith('.zip'):
        continue
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall()
    zip_ref.close()

mkdir: cannot create directory ‘/root/.kaggle’: File exists
usage: kaggle config [-h] {view,set,unset} ...
kaggle config: error: argument command: invalid choice: 'path' (choose from 'view', 'set', 'unset')
POS_CASH_balance.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
application_test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
previous_application.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
installments_payments.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
credit_card_balance.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
bureau_balance.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
HomeCredit_columns_description.csv: Skipping, found more recently modified local copy (use --force to force download)
application_train

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

In [ ]:
train = pd.read_csv('application_train.csv')

train_numeric = train.select_dtypes([np.number]).fillna(0)

X = train_numeric.drop(labels=['TARGET'], axis="columns")
y = train_numeric['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# reg = LogisticRegression(max_iter=1000000).fit(X_train, y_train)
gb = XGBClassifier(tree_method='gpu_hist').fit(X_train, y_train)

In [ ]:
print(accuracy_score(y_test, gb.predict(X_test)))
print(precision_score(y_test, gb.predict(X_test)))
print(recall_score(y_test, gb.predict(X_test)))
print(roc_auc_score(y_test, gb.predict_proba(X_test)[:, 1]))

0.9195811586426679
0.56
0.002828854314002829
0.7436057604734183


In [ ]:
bureau = pd.read_csv('bureau.csv')

bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [ ]:
# Считаем активные и закрытые кредиты.
cnt_active_loans = bureau[bureau.CREDIT_ACTIVE == 'Active'].groupby('SK_ID_CURR', as_index=False)['CREDIT_ACTIVE'].count()
cnt_closed_loans = bureau[bureau.CREDIT_ACTIVE == 'Closed'].groupby('SK_ID_CURR', as_index=False)['CREDIT_ACTIVE'].count()

In [ ]:
# Мерджим наши новые фичи

X_train = X_train.merge(cnt_active_loans, on = 'SK_ID_CURR', how = 'left').fillna(0)
X_test = X_test.merge(cnt_active_loans, on = 'SK_ID_CURR', how = 'left').fillna(0)

X_train = X_train.merge(cnt_closed_loans, on = 'SK_ID_CURR', how = 'left').fillna(0)
X_test = X_test.merge(cnt_closed_loans, on = 'SK_ID_CURR', how = 'left').fillna(0)

In [ ]:
# Теперь посчитаем для всех числовых колонок статистику по ним, что так же будет
# являтся хорошими фичами для нашей модели.

cnt_numeric_fields_statistics = bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index = False).agg(['min', 'max', 'mean', 'sum'])
cnt_numeric_fields_statistics.reset_index(inplace=True)

In [ ]:
columns = ['SK_ID_CURR']
for column in cnt_numeric_fields_statistics.columns.levels[0]:
    if column != 'SK_ID_CURR':
        for stat in cnt_numeric_fields_statistics.columns.levels[1][:-1]:
            columns.append('bureau_%s_%s' % (column, stat))

cnt_numeric_fields_statistics.columns = columns

In [ ]:
X_train = X_train.merge(cnt_numeric_fields_statistics, on = 'SK_ID_CURR', how = 'left').fillna(0)
X_test = X_test.merge(cnt_numeric_fields_statistics, on = 'SK_ID_CURR', how = 'left').fillna(0)

In [ ]:
def one_hot_encoder(df):
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    return pd.get_dummies(df, columns=categorical_columns, dummy_na=True)

In [ ]:
credit_type_ohe = one_hot_encoder(bureau)

credit_type_ohe = credit_type_ohe.groupby('SK_ID_CURR', as_index=False).count()

X_train = X_train.merge(credit_type_ohe, on = 'SK_ID_CURR', how = 'left').fillna(0)
X_test = X_test.merge(credit_type_ohe, on = 'SK_ID_CURR', how = 'left').fillna(0)

In [ ]:
gb_new = XGBClassifier(tree_method='gpu_hist').fit(X_train, y_train)

In [ ]:
print(accuracy_score(y_test, gb_new.predict(X_test)))
print(precision_score(y_test, gb_new.predict(X_test)))
print(recall_score(y_test, gb_new.predict(X_test)))
print(roc_auc_score(y_test, gb_new.predict_proba(X_test)[:, 1]))

0.9195974180121295
0.5909090909090909
0.0026267932915740554
0.7468790050494389
